In [190]:
import pandas as pd
import math
import wget
import matplotlib.pyplot as plt
from matplotlib import animation

#PARSING PARAMETERS & CONTROL
    #init_year -> scan .cnss files starting with given year
    #end_year  -> scan .cnss files up to, but not including, this year (use 2017)
    #init_month-> scan .cnss files starting with given month
    #end_month -> scan .cnss files up to, but not including, this month (use 13)
    #location  -> scan .cnss files and parse out lines that contain the given location parameter, creates 'location'.csv
    #pToCSV    -> Boolean value, when set to False .cnss files will not be parsed into a csv
    #pAvgTable -> Boolean value, when set to False generated .csv files will not be averaged
    
init_year = 1990
end_year = 2017
init_month = 1
end_month = 13

#
p_toCSV = False    #Keep 'False' when testing functions, this will only work when 'eqData' directory exists and contains the .cnss files
p_avgTable = False #Keep 'False'
dl_csvData = False #Triggers to False once data has been downloaded, this prevents duplicate downloads

location = "CI"

In [178]:
#Download CSV data from Github
if dl_csvData == True:
    url = 'https://raw.githubusercontent.com/nevelo/quake-predict/master/m_combined.csv'
    fd = wget.download(url)
    dl_csvData = False

In [179]:
def parseToCSV(directory,loc):
    fpath = str("eqData/" + directory)
    fo = open(fpath)

    #newFile = open(loc + '.csv', 'a')
    newFile = open('combined.csv', 'a')
    
    with fo as f:
        for line in f:
            if location not in line:
                continue
            else:
                date_year = ""
                date_month = ""
                date_day = ""
                mag_str = ""
                
                mag = 0
                pwr = 0
                ct = 0                
                
                #scrape year, month, date
                #for y in range (5,24):
                #    date_str+= line[y]
                for y in range (5,9):
                    date_year+= line[y]
                for m in range (9,11):
                    date_month+= line[m]
                for d in range (11, 13):
                    date_day+= line[d]
                #scrape magnitude    
                for mg in range (130,134):
                    mag_str+= line[mg]
                
                if mag_str[0] == ' ':
                    continue
                
                mag = float(mag_str)
                pwr = (10**(3*mag/2))
                
                if mag > 5.0: 
                    ct = 1
                else:
                    ct = 0                
                
                newFile.write(date_year + ',' + date_month + ',' + date_day + ',' + str(mag) + ',' + str(pwr) + ',' + str(ct) + '\n')
    
    fo.close()
    return

In [180]:
def manCSV(directory):
    #csv_path = str(directory + ".csv")
    csv_path = str('combined.csv')

    IDs = ['year', 'month', 'day', 'mag', 'pwr', 'ct']
    
    df = pd.read_csv(csv_path, names=IDs).groupby(['year', 'month', 'day'])
    
    
    grouped = pd.DataFrame(df.agg(['sum', 'mean']).join(pd.DataFrame(df.size(), 
                                columns=['counts'])))
    #print(grouped)
    grouped.to_csv("m_" + csv_path, sep=',')
    
   
    return grouped

In [181]:
#Create .csv file with given range of year and months
if p_toCSV == True:
    for x in range(init_year, end_year):
        for y in range(init_month, end_month):
            if y<10:
                directory = str(x) + '.0' + str(y) + '.cnss'
            else:
                directory = str(x) + '.' + str(y) + '.cnss'
        
            parseToCSV(directory, location)     

In [182]:
#Parse generated .csv file to average out to daily magnitude, power and count of eq > 5.0
if p_avgTable == True:
    manCSV(location)

In [186]:
import urllib
from mpl_toolkits.basemap import Basemap
import numpy as np

# -> http://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php
feed = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/"

# Significant earthquakes in the last 30 days
# url = urllib.request.urlopen(feed + "significant_month.csv")

# Magnitude > 4.5
#url = str(feed + "4.5_month.csv")

# Magnitude > 2.5
url = str(feed + "2.5_month.csv")

# Magnitude > 1.0
# url = str(feed + "1.0_month.csv")

# Reading and storage of data
data = wget.download(url)
data = data.split(b'\n')[+1:-1]
E = np.zeros(len(data), dtype=[('position',  float, 2),
                               ('magnitude', float, 1)])

for i in range(len(data)):
    row = data[i].split(',')
    E['position'][i] = float(row[2]),float(row[1])
    E['magnitude'][i] = float(row[4])

In [187]:
fig = plt.figure(figsize=(14,10))
ax = plt.subplot(1,1,1)

earth = Basemap(projection='mill')


earth.drawcoastlines(color='0.50', linewidth=0.25)
earth.fillcontinents(color='0.95')

In [192]:
P = np.zeros(50, dtype=[('position', float, 2),
                         ('size',     float, 1),
                         ('growth',   float, 1),
                         ('color',    float, 4)])
scat = ax.scatter(P['position'][:,0], P['position'][:,1], P['size'], lw=0.5,
                  edgecolors = P['color'], facecolors='None', zorder=10)

def update(frame):
    current = frame % len(E)
    i = frame % len(P)

    P['color'][:,3] = np.maximum(0, P['color'][:,3] - 1.0/len(P))
    P['size'] += P['growth']

    magnitude = E['magnitude'][current]
    P['position'][i] = earth(*E['position'][current])
    P['size'][i] = 5
    P['growth'][i]= np.exp(magnitude) * 0.1

    if magnitude < 6:
        P['color'][i]    = 0,0,1,1
    else:
        P['color'][i]    = 1,0,0,1
    scat.set_edgecolors(P['color'])
    scat.set_facecolors(P['color']*(1,1,1,0.25))
    scat.set_sizes(P['size'])
    scat.set_offsets(P['position'])
    return scat,


animation = animation.FuncAnimation(fig, update, interval=10)
plt.show()


ZeroDivisionError: integer division or modulo by zero